In [1]:
import sys
sys.path.append('..')

In [2]:
# notebooks/model_dev.ipynb (or convert to .py later)

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Flatten, Dense
from app.utils.preprocess import load_and_prepare_data

# In your notebook cell

inter_df, num_users, num_articles = load_and_prepare_data(path="../app/data/augmented_articles.csv")

# Build model
user_input = Input(shape=(1,))
article_input = Input(shape=(1,))

user_embedding = Embedding(num_users, 50, input_length=1)(user_input)
article_embedding = Embedding(num_articles, 50, input_length=1)(article_input)

dot_product = Dot(axes=2)([user_embedding, article_embedding])
dot_product = Flatten()(dot_product)
output = Dense(1, activation='linear')(dot_product)

model = Model(inputs=[user_input, article_input], outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train
model.fit(
    [inter_df['user'], inter_df['article']],
    inter_df['interaction'],
    epochs=5,
    batch_size=32,
    validation_split=0.1
)

# Save
model.save("../ml_models/nn_model.h5")


Loading dataset from: c:\Users\tusha\OneDrive\Desktop\news_ml\app\data\augmented_articles.csv


c:\Users\tusha\OneDrive\Desktop\news_ml\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 11.6834 - mae: 3.1095 - val_loss: 10.6713 - val_mae: 2.9851
Epoch 2/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 11.4099 - mae: 3.0578 - val_loss: 10.5896 - val_mae: 2.9714
Epoch 3/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 11.5042 - mae: 3.0678 - val_loss: 10.5089 - val_mae: 2.9579
Epoch 4/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 11.6489 - mae: 3.0861 - val_loss: 10.4286 - val_mae: 2.9443
Epoch 5/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 10.6917 - mae: 2.9524 - val_loss: 10.3494 - val_mae: 2.9309


In [ ]:
# This goes near the top of your notebook/script
from app.utils.preprocess import load_articles
from app.utils.preprocess import load_and_prepare_data
# Load and prepare article DataFrame with temporary IDs
articles_df = load_articles()
articles_temp_df = articles_df.copy()
articles_temp_df['article_id'] = range(1, len(articles_temp_df) + 1)
inter_df, num_users, num_articles = load_and_prepare_data(path="../app/data/augmented_articles.csv")
# ID mapping dictionaries
id_to_Link = dict(zip(articles_temp_df['article_id'], articles_temp_df['Link']))
Link_to_id = dict(zip(articles_temp_df['Link'], articles_temp_df['article_id']))




Loading dataset from: c:\Users\tusha\OneDrive\Desktop\news_ml\app\data\augmented_articles.csv


In [7]:
from app.services.collab_filter import ItemBasedRecommender

# Create recommender and fit it with article_id as key
recommender = ItemBasedRecommender()
recommender.fit(inter_df, articles_temp_df, id_column='article_id')



In [12]:
articles_temp_df[['article_id', 'Link']].head(15)


,article_id,Link
0,1,https://www.bbc.com/news/articles/cjewne81lq4o
1,2,https://www.bbc.com/news/articles/cjewne81lq4o
2,3,https://www.bbc.com/news/articles/cjewne81lq4o
3,4,https://www.bbc.com/news/articles/cjewne81lq4o
4,5,https://www.bbc.com/news/articles/cjewne81lq4o
5,6,https://www.bbc.com/news/articles/cjewne81lq4o
6,7,https://www.bbc.com/news/articles/cjewne81lq4o
7,8,https://www.bbc.com/news/articles/cjewne81lq4o
8,9,https://www.bbc.com/news/articles/cjewne81lq4o
9,10,https://www.bbc.com/news/articles/cjewne81lq4o


In [14]:
# Choose a sample article (say, first one)
sample_article_id = articles_temp_df['article_id'].iloc[0]

# Recommend similar articles
recommended_ids = recommender.recommend_similar_items(sample_article_id, top_n=5)

# Convert back to URLs/titles for interpretation


print("All keys in id_to_Link:", list(id_to_Link.keys())[:10])
print("All recommended_ids:", recommended_ids)

recommended_urls = [id_to_Link[rid] for rid in recommended_ids if rid in id_to_Link]
recommended_urls


All keys in id_to_Link: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
All recommended_ids:                                                 Title  \
15  UK edges towards post-Brexit youth visa deal w...   
22  Student killed in French school stabbing attac...   
53  DRC government and M23 agree to halt fighting ...   
54  DRC government and M23 agree to halt fighting ...   
58  DRC government and M23 agree to halt fighting ...   

                                              Content            Published  \
15  The government is no longer ruling out a youth...  2025-04-25 05:56:01   
22  One student has been killed and at least three...  2025-04-24 20:17:00   
53  Both sides say they have resolved to end confl...  2025-04-24 15:54:37   
54  Both sides say they have resolved to end confl...  2025-04-25 01:31:37   
58  Both sides say they have resolved to end confl...  2025-04-24 13:08:37   

          Source                                               Link  \
15      BBC News     https://www.bbc.com

[]

In [15]:
print(recommended_ids)


                                                Title  \
15  UK edges towards post-Brexit youth visa deal w...   
22  Student killed in French school stabbing attac...   
53  DRC government and M23 agree to halt fighting ...   
54  DRC government and M23 agree to halt fighting ...   
58  DRC government and M23 agree to halt fighting ...   

                                              Content            Published  \
15  The government is no longer ruling out a youth...  2025-04-25 05:56:01   
22  One student has been killed and at least three...  2025-04-24 20:17:00   
53  Both sides say they have resolved to end confl...  2025-04-24 15:54:37   
54  Both sides say they have resolved to end confl...  2025-04-25 01:31:37   
58  Both sides say they have resolved to end confl...  2025-04-24 13:08:37   

          Source                                               Link  \
15      BBC News     https://www.bbc.com/news/articles/c9qw58r0x0do   
22      BBC News     https://www.bbc.com/news/